**El coeficiente de correlacion de Pearson**
-----------------
-----------------
**Definicion**

El coeficiente de correlación de Pearson, también conocido como coeficiente de correlación producto-momento de Pearson, es una medida estadística que cuantifica la
intensidad y dirección de la relación lineal entre dos variables cuantitativas. Matemáticamente, se denota como 𝑟 y toma valores en el intervalo [-1, 1].

El coeficiente de correlación de Pearson entre dos variables 𝑋 y 𝑌 se define como: Cov(𝑋𝑌)/(σX.σ𝑌)
- Cov(𝑋𝑌) es la covarianza entre 𝑋 y 𝑌
- σX es la desviación estandart de X
- σ𝑌 es la desviación estandart de Y

La definicion formal del coeficiente de correlacion de Pearson es:

$r = \frac{\sum_{i=1}^{n} (X_i - \bar{X})(Y_i - \bar{Y})}{\sqrt{\sum_{i=1}^{n} (X_i - \bar{X})^2 \sum_{i=1}^{n} (Y_i - \bar{Y})^2}} \$
-------------------------------------------------------------------------------------------------------------------------------------

donde:
- $n$ es el número de pares de datos.
- $X_i$ e $Y_i$ son los valores individuales de las variables  $X$ e $Y$.
- $\bar{X}$ y $\bar{Y}$ son las medias de $X$ e $Y$, respectivamente.

**Interpretacion de sus resultados**

$r$=1: Indica una correlación positiva perfecta. A medida que 𝑋 aumenta, 𝑌 también aumenta de manera proporcional.

$r$=-1: Indica una correlación negativa perfecta. A medida que 𝑋 aumenta, 𝑌 disminuye de manera proporcional.

$r$=0: Indica que no hay una correlación lineal entre 𝑋 e 𝑌.




Cargamos las librerias y bases de datos
---------------------

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar los datos
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")


Revisamos rápidamente que todo esté funcionando bien
----------

In [24]:
# Exploración de los datos
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [25]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Creamos una matriz de usuarios y peliculas
--------------

In [11]:
# Crear la matriz de usuarios y películas
user_movie_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
user_movie_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Buscamos las 10 peliculas mejor calificadas por el usuario 4
------------

Seleccionar las 10 peliculas mejor calificadas por el usuario 4 es una operacion arbitraria. Podriamos haber seleccionado las Top5 o las Top20. 

Si quisieramos complejizar mas el problema, podriamos tomar la ultima calificacion que tenga 4 o mas estrellas y recomendarle peliculas similares a esa.

A los fines de ejemplificar el uso del coeficiente de correlacion de pearson, utilizaremos un ejemplo mas sencillo.

In [3]:
# Filtrar las calificaciones del usuario 4
user_4_ratings = ratings[ratings['userId'] == 4]

# Ordenar las calificaciones de manera descendente
user_4_ratings_sorted = user_4_ratings.sort_values(by='rating', ascending=False)

# Seleccionar las 10 primeras películas
top_10_user_4_ratings = user_4_ratings_sorted.head(10)

# Hacer un merge con el DataFrame movies para obtener los títulos y géneros
top_10_user_4_movies = top_10_user_4_ratings.merge(movies, on='movieId')

# Seleccionar solo las columnas 'title', 'genres' y 'rating'
top_10_user_4_movies_info = top_10_user_4_movies[['movieId','title', 'genres', 'rating']]

# Mostrar el resultado
top_10_user_4_movies_info

,movieId,title,genres,rating
0,4967,No Man's Land (2001),Drama|War,5.0
1,898,"Philadelphia Story, The (1940)",Comedy|Drama|Romance,5.0
2,1203,12 Angry Men (1957),Drama,5.0
3,3508,"Outlaw Josey Wales, The (1976)",Action|Adventure|Drama|Thriller|Western,5.0
4,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,5.0
5,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,5.0
6,1188,Strictly Ballroom (1992),Comedy|Romance,5.0
7,2599,Election (1999),Comedy,5.0
8,1103,Rebel Without a Cause (1955),Drama,5.0
9,1086,Dial M for Murder (1954),Crime|Mystery|Thriller,5.0


Recomendamos películas similares a lo ya visto
------------

In [4]:
# Obtener el ID de la primera película del top10
target_movie_id = top_10_user_4_movies_info.iloc[0]['movieId']
target_movie_id

4967

No podemos aplicar el coeficiente de correlacion de pearson entre dos peliculas, si una de ellas tiene muy pocas calificaciones. Tenemos que eliminar esas columnas.

El efecto positivo de esta operacion: nos quedamos con una matriz más reducida y esto el calculo del coeficiente de correlacion de pearson va a ser computacionalmente menos costoso.

El efecto negativo de esta operacion: las peliculas con menos de 5 recomendaciones no van a formar parte de la lista de items recomendados. Si quisieramos recomendar peliculas que no son tan populares y no marcan tendencia, tendriamos que combinar este procedimiento con otros algoritmos de recomendacion

In [19]:
# Contar el número de calificaciones por película
rating_counts = user_movie_matrix.notna().sum()

# Filtrar las columnas que tenga menos de 5 calificaciones
valid_movies = rating_counts[rating_counts >= 5].index
user_movie_matrix_reduced = user_movie_matrix[valid_movies]


La matriz original era de 610 filas × 9724 columnas. Ahora vamos a tener una matriz reducida

In [6]:
print("Matriz de: ", user_movie_matrix_reduced.shape)

Matriz de:  (610, 3650)


El siguiente paso es calcular el coeficiente de correlación de pearson entre la pelicula objetivo y todas aquellas peliculas que:
1) Tengan mas de 5 calificaciones (por el paso anterior)
2) Como minimo, hayan sido calificadas por 3 usuarios que tambien calificaron a la pelicula objetivo
3) Tengan una desviacion estandar superior a 0

La justificacion para implementar el paso 2: las correlaciones calculadas a partir de muestras muy pequeñas (menos de 3) son poco confiables y pueden ser muy sensibles a valores atípicos. Necesitar al menos 3 puntos de datos proporciona una mínima base para un cálculo más estable.

La justificacion para implementar el paso 3: dado que la desviación estándar mide la dispersión de los datos, una desviación estándar de cero indica que no hay variabilidad en las calificaciones. En términos de la fórmula de la correlación de Pearson, esto significa que el denominador será cero, resultando en un NaN.

In [20]:
# Obtener la columna de la película objetivo
target_movie_ratings = user_movie_matrix_reduced[target_movie_id]

# Calcular el coeficiente de correlación de Pearson
correlation_dict = {}
for movie_id in user_movie_matrix_reduced.columns:
    if movie_id == target_movie_id:
        continue

    # Obtener las calificaciones para la película comparada
    other_movie_ratings = user_movie_matrix_reduced[movie_id]
    
    # Verificar que ambas columnas tengan al menos 3 calificaciones en común
    common_ratings = user_movie_matrix_reduced[[target_movie_id, movie_id]].dropna()
    if len(common_ratings) >= 3:
        # Verificar las estadísticas de las calificaciones comunes
        stats = common_ratings.describe()
        if stats.loc['std', target_movie_id] == 0 or stats.loc['std', movie_id] == 0:
            #Podemos usar esta linea para mostrar que peliculas eliminamos de la lista por tener desv estandar = 0
            #print(f"Skipping movie_id: {movie_id} due to zero standard deviation in common ratings")
            continue

        # Calcular la correlación de Pearson
        correlation = common_ratings.corr().iloc[0, 1]
        if not np.isnan(correlation):  # Verificar si la correlación no es NaN
            correlation_dict[movie_id] = correlation
        else:
            print(f"NaN correlation for movie_id: {movie_id} with target_movie_id: {target_movie_id}")
            print(common_ratings)
            print("Statistics for common ratings:")
            print(stats)
            
# Convertir el diccionario a un DataFrame para una mejor visualización y ordenar desendentemente
correlation_df = pd.DataFrame(list(correlation_dict.items()), columns=['movieId', 'pearson_correlation'])
correlation_df = correlation_df.sort_values(by='pearson_correlation', ascending=False)



In [18]:
# Merge con el DataFrame movies para obtener títulos y géneros
recommended_movies = correlation_df.merge(movies[['movieId', 'title', 'genres']], on='movieId')

# Seleccionar solo las columnas 'movieId', 'title', 'genres', y 'pearson_correlation'
recommended_movies_info = recommended_movies[['movieId', 'title', 'genres', 'pearson_correlation']]

print("Peliculas similares a la pelicula objetivo")
recommended_movies_info

Peliculas similares a la pelicula objetivo


,movieId,title,genres,pearson_correlation
0,3262,Twin Peaks: Fire Walk with Me (1992),Crime|Drama|Mystery|Thriller,1.0
1,69844,Harry Potter and the Half-Blood Prince (2009),Adventure|Fantasy|Mystery|Romance|IMAX,1.0
2,91500,The Hunger Games (2012),Action|Adventure|Drama|Sci-Fi|Thriller,1.0
3,18,Four Rooms (1995),Comedy,1.0
4,7099,Nausicaä of the Valley of the Wind (Kaze no ta...,Adventure|Animation|Drama|Fantasy|Sci-Fi,1.0
...,...,...,...,...
897,4370,A.I. Artificial Intelligence (2001),Adventure|Drama|Sci-Fi,-1.0
898,2001,Lethal Weapon 2 (1989),Action|Comedy|Crime|Drama,-1.0
899,3450,Grumpy Old Men (1993),Comedy,-1.0
900,54272,"Simpsons Movie, The (2007)",Animation|Comedy,-1.0


Vamos a eliminar todas las peliculas que ya fueron vistas por el usuario 4, para recomendarle peliculas nuevas

In [12]:
# Obtener los movieId de las películas ya calificadas por el usuario 4
seen_movie_ids = user_4_ratings['movieId'].tolist()

# Eliminar las películas ya vistas por el usuario 4 de las recomendaciones
filtered_recommendations = recommended_movies_info[~recommended_movies_info['movieId'].isin(seen_movie_ids)]
filtered_recommendations

,movieId,title,genres,pearson_correlation
0,3262,Twin Peaks: Fire Walk with Me (1992),Crime|Drama|Mystery|Thriller,1.0
1,69844,Harry Potter and the Half-Blood Prince (2009),Adventure|Fantasy|Mystery|Romance|IMAX,1.0
2,91500,The Hunger Games (2012),Action|Adventure|Drama|Sci-Fi|Thriller,1.0
3,18,Four Rooms (1995),Comedy,1.0
4,7099,Nausicaä of the Valley of the Wind (Kaze no ta...,Adventure|Animation|Drama|Fantasy|Sci-Fi,1.0
...,...,...,...,...
897,4370,A.I. Artificial Intelligence (2001),Adventure|Drama|Sci-Fi,-1.0
898,2001,Lethal Weapon 2 (1989),Action|Comedy|Crime|Drama,-1.0
899,3450,Grumpy Old Men (1993),Comedy,-1.0
900,54272,"Simpsons Movie, The (2007)",Animation|Comedy,-1.0


Nos quedamos con las top10 peliculas no vistas por el usuario 4

In [13]:
top_10_recommendations = filtered_recommendations.head(10)
top_10_recommendations

,movieId,title,genres,pearson_correlation
0,3262,Twin Peaks: Fire Walk with Me (1992),Crime|Drama|Mystery|Thriller,1.0
1,69844,Harry Potter and the Half-Blood Prince (2009),Adventure|Fantasy|Mystery|Romance|IMAX,1.0
2,91500,The Hunger Games (2012),Action|Adventure|Drama|Sci-Fi|Thriller,1.0
3,18,Four Rooms (1995),Comedy,1.0
4,7099,Nausicaä of the Valley of the Wind (Kaze no ta...,Adventure|Animation|Drama|Fantasy|Sci-Fi,1.0
5,40278,Jarhead (2005),Action|Drama|War,1.0
6,6755,Bubba Ho-tep (2002),Comedy|Horror,1.0
7,4776,Training Day (2001),Crime|Drama|Thriller,1.0
8,101,Bottle Rocket (1996),Adventure|Comedy|Crime|Romance,1.0
10,5254,Blade II (2002),Action|Horror|Thriller,1.0


Adicional: verificar que recomendamos peliculas no vistas anteriormente
-----------------

In [14]:
# Paso 1: Extraer los movieId de top_10_recommendations
top_10_movie_ids = top_10_recommendations['movieId'].tolist()

# Paso 2: Verificar que esas columnas en user_movie_matrix no tengan calificaciones en la fila 4
user_4_ratings = user_movie_matrix.loc[4, top_10_movie_ids]

In [15]:
user_4_ratings

movieId
3262    NaN
69844   NaN
91500   NaN
18      NaN
7099    NaN
40278   NaN
6755    NaN
4776    NaN
101     NaN
5254    NaN
Name: 4, dtype: float64